In [25]:
import easyocr
import cv2
import sqlite3
import pandas as pd
import os
import re

path = 'menu.jpg'

if not os.path.exists(path):
    raise FileNotFoundError(f"The file {path} does not exist.")

reader = easyocr.Reader(['en'])

try:
    img = cv2.imread(path)
    if img is None:
        raise ValueError("The image file could not be read.")
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = reader.readtext(img_rgb)
    
    extracted_text = ""
    for detection in result:
        extracted_text += detection[1] + "\n"
    
    print(extracted_text)

except Exception as e:
    print(f"An error occurred: {e}")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


al
BAY
CHINATOWN
by Nadle Bar
CA
Blwe
Dilli Papri Chaat
60
Spring Rolls
110
ny
Paani Puri
60
Veg: Dimsums
110
al
Cottage Cheese Tacos
110
Chicken Spring Rolls
125
Pav Bhaji
110
Chicken Lollypop
125
Mexican Immigrants Burger
110
Chicken Dimsums
125
Big Boss Burger
110
Nachos with Cheese Sauce
125
COPPER
CHIMNEY
Corn & Cheese Balls
125
Indi Paneer Pizza
145
Dal Makhani
90
Channa Bhatura
155
Paneer Tikka Roll
165
Macaroni Cream & Cheese
155
Biryani
165
Penne Mexicana
155
Paneer Tikka (4pcs)
185
Texan Pizza
145
Paneer Makhani
205
Bhatura
25
Chicken Biryani

185
Pav
25
Reshmi Kebab Roll
205
Iced Tea (Peach, Lemon_
Strawberry)  85
Murg Makhani
225
30
Reshmi Kebab (4pcs)
225
Nquana
MRP
Roomali Roti
45
"Conditions Apply:
a
a
a
nii
a
Image ID: DHIGAE
alamy
WWW
alamycom
Veg:
Veg;



In [26]:
lines = extracted_text.split("\n")
menu_dict = {}
price_pattern = re.compile(r'(\d+)$')

current_item = ""
for line in lines:
    match = price_pattern.search(line)
    if match:
        item = line[:match.start()].strip()
        price = int(match.group())
        if current_item:
            item = current_item + " " + item
            current_item = ""
        if item:
            menu_dict[item] = price
    else:
        if line.strip():
            if current_item:
                current_item += " " + line.strip()
            else:
                current_item = line.strip()

print(menu_dict)

{'al BAY CHINATOWN by Nadle Bar CA Blwe Dilli Papri Chaat ': 60, 'Spring Rolls ': 110, 'ny Paani Puri ': 60, 'Veg: Dimsums ': 110, 'al Cottage Cheese Tacos ': 110, 'Chicken Spring Rolls ': 125, 'Pav Bhaji ': 110, 'Chicken Lollypop ': 125, 'Mexican Immigrants Burger ': 110, 'Chicken Dimsums ': 125, 'Big Boss Burger ': 110, 'Nachos with Cheese Sauce ': 125, 'COPPER CHIMNEY Corn & Cheese Balls ': 125, 'Indi Paneer Pizza ': 145, 'Dal Makhani ': 90, 'Channa Bhatura ': 155, 'Paneer Tikka Roll ': 165, 'Macaroni Cream & Cheese ': 155, 'Biryani ': 165, 'Penne Mexicana ': 155, 'Paneer Tikka (4pcs) ': 185, 'Texan Pizza ': 145, 'Paneer Makhani ': 205, 'Bhatura ': 25, 'Chicken Biryani ': 185, 'Pav ': 25, 'Reshmi Kebab Roll ': 205, 'Iced Tea (Peach, Lemon_ Strawberry)': 85, 'Murg Makhani ': 225, 'Reshmi Kebab (4pcs) ': 225, 'Nquana MRP Roomali Roti ': 45}


In [27]:
conn = sqlite3.connect('menu.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS menu (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    item TEXT NOT NULL,
    price INTEGER NOT NULL
)
''')

for item, price in menu_dict.items():
    cursor.execute('INSERT INTO menu (item, price) VALUES (?, ?)', (item, price))

conn.commit()
conn.close()

print("Data has been successfully inserted into the database.")

Data has been successfully inserted into the database.


In [32]:
conn = sqlite3.connect('menu.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM menu')
data = cursor.fetchall()

conn.close()

df = pd.DataFrame(data, columns=['id', 'item', 'price'])
df.tail(5)

,id,item,price
57,58,Reshmi Kebab Roll,205
58,59,"Iced Tea (Peach, Lemon_ Strawberry)",85
59,60,Murg Makhani,225
60,61,Reshmi Kebab (4pcs),225
61,62,Nquana MRP Roomali Roti,45
